In [1]:
import os
# proxy
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'

In [2]:
# test google
import requests
response = requests.get("https://www.google.com")

In [3]:
import datasets

In [6]:
train_dataset = datasets.load_dataset("iwslt2017", "iwslt2017-en-fr", split='train')
val_dataset = datasets.load_dataset("iwslt2017", "iwslt2017-en-fr", split='validation')

Generating train split:   0%|          | 0/232825 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8597 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/890 [00:00<?, ? examples/s]

In [7]:
train_dataset[0]

{'translation': {'en': "Thank you so much, Chris. And it's truly a great honor to have the opportunity to come to this stage twice; I'm extremely grateful.",
  'fr': "Merci beaucoup, Chris. C'est vraiment un honneur de pouvoir venir sur cette scène une deuxième fois. Je suis très reconnaissant."}}

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-en")

In [6]:
# 遍历 dataset，获取最长句子 tokenization 后的最长长度
from tqdm import tqdm
max_length = 0
# for example in tqdm(train_dataset):
#     tokens = tokenizer.encode(example['translation']['fr'])
#     max_length = max(max_length, len(tokens))
for example in tqdm(val_dataset):
    tokens = tokenizer.encode(example['translation']['fr'])
    max_length = max(max_length, len(tokens))
print(max_length)

100%|██████████| 3000/3000 [00:00<00:00, 9379.19it/s]

144


In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

In [8]:
# 遍历 dataset，获取最长句子 tokenization 后的最长长度
from tqdm import tqdm
max_length = 0
# for example in tqdm(train_dataset):
#     tokens = tokenizer.encode(example['translation']['en'])
#     max_length = max(max_length, len(tokens))
for example in tqdm(val_dataset):
    tokens = tokenizer.encode(example['translation']['en'])
    max_length = max(max_length, len(tokens))
print(max_length)

100%|██████████| 3000/3000 [00:00<00:00, 10772.76it/s]

118


In [3]:
from data.dataset import get_dataloader
train_loader = get_dataloader("train", 64)

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/30 [00:00<?, ?it/s]

In [4]:
x, y = next(iter(train_loader))

In [6]:
from transformers import AutoTokenizer
fr_en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-en")
en_fr_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

In [7]:
enc = en_fr_tokenizer("123")
enc

{'input_ids': [12353, 0], 'attention_mask': [1, 1]}

In [8]:
enc = en_fr_tokenizer.encode("123")
enc

[12353, 0]

In [40]:
print(x[0])
print(y[0])

Bien que le requérant demande une indemnisation en dollars des États-Unis, les dispositions financières des accords de crédit-bail étaient libellées en dinars koweïtiens et Santa Fe a calculé son manque à gagner dans cette dernière monnaie.
Although the claimant seeks compensation in United States dollars, the payment provisions of the lease agreements were stated in Kuwaiti dinars, and Santa Fe has calculated its claim for loss of revenue in that currency.


In [42]:
print(fr_en_tokenizer.bos_token_id)
print(fr_en_tokenizer.eos_token_id)
print(fr_en_tokenizer.pad_token_id)
fr_en_tokenizer.add_special_tokens({'bos_token': '<bos>'})
print(fr_en_tokenizer.bos_token_id)

59514
0
59513
59514


In [55]:
encoded = fr_en_tokenizer(x[0], return_tensors='pt', padding=True, truncation=True, max_length=512) 
print(encoded)

{'input_ids': tensor([[ 1533,    29,    19,  8483,   434,    38, 14174,    23,   678,    13,
           206,    21,  1014,     2,    16,   710,  2164,    13,  2617,     5,
          2692,    21,  3227,   107,   736, 11332,   114,    23,    20,   190,
         10141, 32206,     9,    11,  6979,  8932,    15, 14338,   113,  3104,
            17,  6893,    31,   120,  1596,  9001,     3,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [67]:
print(en_fr_tokenizer.bos_token_id)
print(en_fr_tokenizer.eos_token_id)
print(en_fr_tokenizer.pad_token_id)
en_fr_tokenizer.add_special_tokens({'bos_token': '<bos>'})
print(en_fr_tokenizer.bos_token_id)

59514
0
59513
59514


In [74]:
encoded = en_fr_tokenizer(y[0], return_tensors='pt', padding=True, truncation=True, max_length=512, add_special_tokens=False)
print(encoded)

{'input_ids': tensor([[ 3237,     4, 13663, 11512,  3401,    18,   185,   194,   678,     2,
             4,  2688,   968,     7,     4, 14859,  2224,   128,  2645,    18,
         31183,  2111, 12766,     9,     2,    10,  6979,  8932,    94,  7193,
            96,  3580,    26,  3070,     7,  7427,    18,    33,  7441,     3]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [75]:
print(en_fr_tokenizer.decode(encoded['input_ids'][0]))

Although the claimant seeks compensation in United States dollars, the payment provisions of the lease agreements were stated in Kuwaiti dinars, and Santa Fe has calculated its claim for loss of revenue in that currency.


In [87]:
encoded = en_fr_tokenizer("<bos>"+y[0]+"</s>"+"<pad>", return_tensors='pt', padding=True, truncation=True, max_length=512, add_special_tokens=False)
print(encoded)

{'input_ids': tensor([[59514,  3237,     4, 13663, 11512,  3401,    18,   185,   194,   678,
             2,     4,  2688,   968,     7,     4, 14859,  2224,   128,  2645,
            18, 31183,  2111, 12766,     9,     2,    10,  6979,  8932,    94,
          7193,    96,  3580,    26,  3070,     7,  7427,    18,    33,  7441,
             3,     0, 59513]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [89]:
print(en_fr_tokenizer.eos_token)

</s>


In [2]:
from data.dataset import get_dataloader
from tokenizer.fr_en_tokenizer import Tokenizer

In [ ]:
train_loader = get_dataloader("train", 4, shuffle=False)

In [6]:
batch = next(iter(train_loader))
x, y = batch
print(x)
print(y)

('Reprise de la session', 'Je déclare reprise la session du Parlement européen qui avait été interrompue le vendredi 17 décembre dernier et je vous renouvelle tous mes vux en espérant que vous avez passé de bonnes vacances.', 'Comme vous avez pu le constater, le grand "bogue de l\'an 2000" ne s\'est pas produit. En revanche, les citoyens d\'un certain nombre de nos pays ont été victimes de catastrophes naturelles qui ont vraiment été terribles.', 'Vous avez souhaité un débat à ce sujet dans les prochains jours, au cours de cette période de session.')
('Resumption of the session', 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.', "Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.", 'You ha

In [3]:
warped_tokenizer = Tokenizer(256)

In [4]:
train_loader2 = get_dataloader("train", 4, shuffle=False, warped_tokenizer=warped_tokenizer)

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/30 [00:00<?, ?it/s]

In [5]:
batch2 = next(iter(train_loader2))
x, y1, y2 = batch2

In [6]:
x, y1, y2

(tensor([[59514,   660, 14717,  ...,     0,     0,     0],
         [59514,   131,  6629,  ...,     0,     0,     0],
         [59514,  1247,    88,  ...,     0,     0,     0],
         [59514,   344,   867,  ...,     0,     0,     0]]),
 tensor([[59514,   660, 10252,  ...,     0,     0,     0],
         [59514,    47, 15845,  ...,     0,     0,     0],
         [59514,  3237,     2,  ...,     0,     0,     0],
         [59514,   213,    79,  ...,     0,     0,     0]]),
 tensor([[  660, 10252,   529,  ...,     0,     0,     0],
         [   47, 15845, 15298,  ...,     0,     0,     0],
         [ 3237,     2,    48,  ...,     0,     0,     0],
         [  213,    79,  2000,  ...,     0,     0,     0]]))

In [7]:
tm = lambda x ,y, z: {"input_ids": x, "labels": y, "decoder_input_ids": z}

In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 定义一个余弦相似度损失类
class CosineSimilarityLoss(nn.Module):
    def __init__(self):
        super(CosineSimilarityLoss, self).__init__()
        self.cosine_sim = nn.CosineSimilarity(dim=1)  # 计算通道维上的余弦相似度

    def forward(self, input1, input2):
        # 计算余弦相似度
        cosine_similarity = self.cosine_sim(input1, input2)
        # 余弦相似度损失为 1 减去平均余弦相似度
        loss = cosine_similarity
        return loss

# 假设 feature_map1 和 feature_map2 是两个形状为 [batch_size, channels, height, width] 的特征图
feature_map1 = torch.tensor([[[[1, 1, 3], [1, 3, 0]], [[1, 1, 4], [1, 3, 1]]]], dtype=float)  # 特征图1
feature_map2 = torch.tensor([[[[2, 1, 0], [2, 6, 1]], [[2, 1, 1], [2, 6, 1]]]], dtype=float)  # 特征图2

# 创建余弦相似度损失实例
cosine_loss = CosineSimilarityLoss()

# 计算两个特征图之间的余弦相似度损失
loss = cosine_loss(feature_map1, feature_map2)
# print("Cosine Similarity Loss:", loss.item())
print(loss)

tensor([[[1.0000, 1.0000, 0.8000],
         [1.0000, 1.0000, 0.7071]]], dtype=torch.float64)
